In [ ]:
import numpy as np, matplotlib.pyplot as plt

%matplotlib inline

from skimage.draw import circle

image = np.zeros((64, 64)).astype('bool')

for k in range(30):
    x0, y0 = np.random.randint(64, size=(2))
    image[circle(x0, y0, 3)] = True

from scipy.ndimage import label

labels, num_features = label(image)

In [ ]:
plt.imshow(labels)
plt.gray()

In [ ]:
from skimage.data import coins
from scipy.ndimage import gaussian_filter

image = gaussian_filter(coins(), sigma=0.5)

plt.hist(image.flatten(), bins=128)
plt.show() 

In [ ]:
level_1 = coins()<=80
level_2 = (coins()>80) * (coins()<=112)
level_3 = (coins()>112) * (coins()<=137)
level_4 = coins()>137

plt.figure(figsize=(24,12))
plt.subplot2grid((2,4), (0,0), colspan=2, rowspan=2)
plt.imshow(coins())
plt.gray()
plt.subplot2grid((2,4),(0,2))
plt.imshow(level_1)
plt.axis('off')
plt.subplot2grid((2,4),(0,3))
plt.imshow(level_2)
plt.axis('off')
plt.subplot2grid((2,4), (1,2))
plt.imshow(level_3)
plt.axis('off')
plt.subplot2grid((2,4), (1,3))
plt.imshow(level_4)
plt.axis('off')
plt.show()

In [ ]:
from scipy.ndimage.morphology import binary_fill_holes

level_4 = binary_fill_holes(level_4)

labels, num_features = label(level_4)

In [ ]:
plt.imshow(labels)

In [ ]:
from scipy.ndimage import watershed_ift

markers_x = [50, 125, 200, 255]
markers_y = [50, 100, 150, 225, 285, 350]

markers = np.zeros_like(image).astype('int16')
markers_index = [[x,y] for x in markers_x for y in markers_y]

for index, location in enumerate(markers_index):
    markers[location[0], location[1]] = index+5

segments = watershed_ift(image, markers)

In [ ]:
plt.imshow(segments)

In [ ]:
from skimage.segmentation import random_walker

segments = random_walker(image, markers)

In [ ]:
plt.imshow(segments)

In [ ]:
markers[level_1] = markers[level_2] = -1

segments = random_walker(image, markers)

In [ ]:
plt.imshow(segments)

In [ ]:
from skimage.feature import canny

edges = canny(coins(), sigma=3.5)

In [ ]:
plt.figure(figsize=(12,6))
plt.subplot(121)
plt.imshow(coins())
plt.subplot(122)
plt.imshow(edges)
plt.show()

In [ ]:
from skimage.draw import line, ellipse_perimeter, circle_perimeter

image = np.zeros((64, 64)).astype('bool')
image[ellipse_perimeter(10, 10, 9, 5)] = True
image[circle_perimeter(30, 35, 15)] = True
image[line(0, 3, 63, 40)] = True

In [ ]:
from skimage.transform import hough_line, hough_line_peaks

Hspace, thetas, distances = hough_line(image)
hough_line_peaks(Hspace, thetas, distances)

In [ ]:
def hesse_line(theta, distance, thickness):
    return lambda i, j: np.abs(distance - np.cos(theta)*j - np.sin(theta)*i) < thickness

peak, theta, r = hough_line_peaks(Hspace, thetas, distances)

detected_lines = np.fromfunction(hesse_line(theta, r, 1.), (64, 64))

In [ ]:
plt.figure(figsize=(24,12))
plt.subplot(131)
plt.imshow(image)
plt.title('Original')
plt.subplot(132)
plt.imshow(Hspace)
plt.title('Hspace - look for peaks!')
plt.subplot(133)
plt.imshow(detected_lines)
plt.title('Lines detected')
plt.show()

In [ ]:
from skimage.transform import hough_circle

detected_circles = hough_circle(image, radius=np.array([15]))

np.where(detected_circles == detected_circles.max())

In [ ]:
from skimage.data import hubble_deep_field
from skimage.feature import blob_doh
from skimage.color import rgb2gray

image = rgb2gray(hubble_deep_field())

blobs = blob_doh(image)

plt.figure(figsize=(12,6))
ax1 = plt.subplot(121)
ax1.imshow(image)
plt.gray()
ax2 = plt.subplot(122)
ax2.imshow(np.zeros_like(image))

for blob in blobs:
    y, x, r = blob
    c = plt.Circle((x, y), r, color='white', lw=1, fill=False)
    ax2.add_patch(c)

plt.show()

In [ ]:
from skimage.feature import corner_fast, corner_orientations, corner_peaks
from skimage.data import text

mask = np.ones((5,5))

corner_response = corner_fast(text(), threshold=0.2)
corner_pos = corner_peaks(corner_response)
corner_orientation = corner_orientations(text(), corner_pos, mask)

for k in range(5):
    y, x = corner_pos[k]
    angle = np.rad2deg(corner_orientation[k])
    print "Corner at ({}, {}) with orientation {}".format(x,y,angle)

In [ ]:
plt.figure(figsize=(12,6))
ax = plt.subplot(111)
ax.imshow(text())
plt.gray()

for corner in corner_pos:
    y, x = corner
    c = plt.Circle((x, y), 2, lw=1, fill=False, color='red')
    ax.add_patch(c)

plt.show()

In [ ]:
from scipy.misc import imread

quixote = imread('quixote.tiff')
bin_quixote = (quixote[:,:,0]<50)
letter_e = quixote[10:29, 250:265]
bin_e = bin_quixote[10:29, 250:265]

from scipy.ndimage.morphology import binary_hit_or_miss

x, y = np.where(binary_hit_or_miss(bin_quixote, bin_e))

plt.figure(figsize=(24,12))
ax = plt.subplot(111)
ax.imshow(bin_quixote)

for loc in zip(y, x):
    c = plt.Circle((loc[0], loc[1]), 15, color='red', fill=False)
    ax.add_patch(c)

plt.show()

In [ ]:
from cv2 import matchTemplate, TM_SQDIFF

detection = matchTemplate(quixote, letter_e, TM_SQDIFF)
x, y = np.where(detection <= detection.mean()/8.)

plt.figure()
ax = plt.subplot(111)
ax.imshow(quixote)

for loc in zip(y, x):
    r = pltRectangle((loc[0], loc[1]), 15, 19, fill=False)
    ax.add_patch(r)

plt.show()

In [ ]:
from cv2 import CascadeClassifier
from skimage.data import lena

face_cascade = CascadeClassifier('haarcascade_frontalface_default.xml')
eye_cascade = CascadeClassifier('haarcascade_eye.xml')

faces = face_cascade.detectMultiScale(lena())
eyes = eye_cascade.detectMultiScale(lena())

print faces
print eyes

In [ ]:
plt.figure()
ax = plt.subplot(111)
ax.imshow(lena())

x, y, w, ell = faces[0]
r = plt.Rectangle((x, y), w, ell, lw=1, fill=False)
ax.add_patch(r)

for eye in eyes:
    x,y,w, ell = eye
    r = plt.Rectangle((x,y),w,ell,lw=1,fill=False)
    ax.add_patch(r)

plt.show()